In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras import backend as K
import src.data_augmentation as da
import src.pre_processing as pp
K.set_image_dim_ordering('th')

In [3]:
from keras.datasets import mnist
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
# turns greyscale into black and white
mnist_x_train = (mnist_x_train>0).astype('float32')
mnist_x_test = (mnist_x_test>0).astype('float32')
# combine training, validation and test dataset
mnist_x_train = np.append(mnist_x_train, mnist_x_test, axis=0)
y_train = np.append(mnist_y_train, mnist_y_test, axis=0)
X_train = mnist_x_train.reshape(mnist_x_train.shape[0], 1, 28, 28)
# grab data generator
datagen = da.get_mnist_image_generator(X_train)

In [9]:
X_train = X_train.reshape(-1, 28, 28)
y_train = y_train.reshape(-1, 1)

In [14]:
validation = pp.get_training()
test = pp.get_test()

In [14]:
X_valid = np.array([x for x, y in validation])
y_valid = np.array([y for x, y in validation])
X_test = test

In [15]:
pixel_count = X_train.shape[1] * X_train.shape[2]

In [16]:
# Normalize values (0-1)
X_train = np.divide(X_train, 255)
X_valid = np.divide(X_valid, 255)
X_test = np.divide(X_test, 255)
y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)

In [17]:
class_count = y_train.shape[1]

In [18]:
# Seed for reproducibility
seed = 42
np.random.seed(seed)

In [20]:
# FOR CNN
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_valid = X_valid.reshape(X_valid.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [16]:
def base_cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [32]:
def larger_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(50, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
def deep_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='linear', input_shape=(1, 28, 28)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1000, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(512, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
cnn_model_larger = larger_cnn()
# cnn_model_larger.fit(X_train, y_train, epochs=1000, batch_size=200, verbose=2)
cnn_model_larger.fit_generator(datagen.flow(X_train, y_train, batch_size=32), use_multiprocessing=True, epochs=100, 
                               validation_data=(X_valid, y_valid), steps_per_epoch=len(X_train))

In [56]:
cnn_large_pred = cnn_model_larger.predict(X_test)

In [57]:
cnn_large_pred_nums = np.array([np.argmax(pred) for pred in cnn_large_pred])

In [58]:
cnn_large_pred_final = np.array([(i, x) for (i, x) in enumerate(cnn_large_pred_nums)])

In [59]:
np.savetxt('../data/test_y_largecnn.csv', cnn_large_pred_final, delimiter=',', fmt='%d', header='Id,Label')